In [1]:
import numpy as np
import astropy.io.fits as fits
import pyklip.fitpsf as fitpsf
import matplotlib.pylab as plt

In [ ]:
sep = 6
pa = 55
length = 2.5  # guess here also
FWHM = 3.61
smooth = None
output_prefix = 'doGaussian/Cont3.61'
# set some boundaries for your MCMC
x_range = 1.5 # in pixels, anywhere from 1.5-5 is reasonable
y_range = 1.5  # same as x
flux_range = [10,200] # flux can vary by an order of magnitude
corr_len_range = 3  # between 0.3 and 30

# get FM frame
fm_frame = fits.getdata(output_prefix + "-fmpsf-KLmodes-all.fits")[0]
fm_header = fits.getheader(output_prefix + "-fmpsf-KLmodes-all.fits")
if smooth:
    print('smoothing!')
    fm_frame = nan_gaussian_filter(fm_frame, smooth)
fm_centx = fm_header['PSFCENTX']
fm_centy = fm_header['PSFCENTY']

# get data_stamp frame
data_frame = fits.getdata(output_prefix + "-klipped-KLmodes-all.fits")[0]
data_header = fits.getheader(output_prefix + "-klipped-KLmodes-all.fits")
if smooth:
    data_frame = nan_gaussian_filter(data_frame, smooth)
data_centx = data_header['PSFCENTX']
data_centy = data_header['PSFCENTY']

plt.imshow(data_frame[200:250,200:250],origin='lower')

# create Planet Evidence Module
fit = fitpsf.PlanetEvidence(sep, pa, 15, output_prefix)
print('created PE module')

# generate FM stamp
# padding should be greater than 0 so we don't run into interpolation problems
fit.generate_fm_stamp(fm_frame, [fm_centx, fm_centy], padding=5)

# generate data_stamp stamp
# not that dr=4 means we are using a 4 pixel wide annulus to sample the noise for each pixel
# exclusion_radius excludes all pixels less than that distance from the estimated location of the planet
fit.generate_data_stamp(data_frame, [data_centx, data_centy], dr=4, exclusion_radius=10)
print('generated FM \& data stamps')
# set kernel, no read noise
corr_len_guess = 3.
corr_len_label = r"$l$"
fit.set_kernel("matern32", [corr_len_guess], [corr_len_label])
print('set kernel')
fit.set_bounds(x_range, y_range, flux_range, [corr_len_range])
print('set bounds')
#Run the pymultinest fit
fit.multifit()
print('ran fit')
global corn, nullcorn
corn, nullcorn = fit.fit_plots()
plt.show()
corn
plt.savefig(output_prefix+'_evidence_corner'+PSFpath.replace('do',str(FWHM))+'.png', transparent=True, dpi=300)
plt.show()
nullcorn
plt.savefig(output_prefix+'_null_corner'+PSFpath.replace('do',str(FWHM))+'.png', transparent=True, dpi=300)
plt.show()

evidence = fit.fit_stats()

#Forward model evidence
fm_evidence = evidence[0]['nested sampling global log-evidence']
#forward model parameter distributions, containing the median and percentiles for each
fm_posteriors = evidence[0]['marginals']

#Null model evidence
null_evidence = evidence[1]['nested sampling global log-evidence']
#null parameter distributions, containing the median and percentiles for each
null_posteriors = evidence[1]['marginals']
global evidence_ratio
evidence_ratio = np.exp(fm_evidence)/np.exp(null_evidence)

print('evidence ratio is: ',round(np.log(evidence_ratio), 4),' >5 is strong evidence')
global residfig, resids, snr
residnfig, snr = fit.fm_residuals()
residfig, resids = residnfig
residfig
plt.savefig(output_prefix+'_BKA_residuals'+PSFpath.replace('do',str(FWHM))+'.png', transparent=True, dpi=300)


created PE module


/home/wbalmer21/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


generated FM \& data stamps
set kernel
set bounds


/home/wbalmer21/anaconda3/lib/python3.8/site-packages/numpy/linalg/linalg.py:2098: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
